> import libraries

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
#convert .ipynb notes to scripts 
import notebooktoall as nb

# predefined module
from prediction_utilities import preprocess, outlier_vars,change_datatype,extract_dates,fill_na

#preprocessing libraries 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
import pandas as pd
import seaborn as sns
import numpy as np 

> import datasets

In [3]:
na_value=['',' ','nan','Nan','NaN','na', 'Na', 'NA', '<Na>'] 
store = pd.read_csv('../../datasets/rossmann-store-sales/store.csv',na_values=na_value)
test = pd.read_csv('../../datasets/rossmann-store-sales/test.csv', na_values=na_value)
train = pd.read_csv('../../datasets/rossmann-store-sales/train.csv', na_values=na_value)

/home/patrick/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
store.head(1)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [5]:
train.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1


In [6]:
test.head(1)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0


Data Cleaning & Preprocessing

first merge train,test and store data

In [7]:
new_train = train.merge(store, on=["Store"], how='inner')
new_test=test.merge(store,on=["Store"], how="inner")

In [8]:
new_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [9]:
new_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [10]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [11]:
new_train.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

In [12]:
new_train.dtypes

Store                          int64
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

> check for outliers

In [13]:
outlier_vars(new_train)

,Sales,Customers,Open,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceYear
0,5263,555,1,1,1270.0,2008.0
1,5020,546,1,1,1270.0,2008.0
2,4782,523,1,1,1270.0,2008.0
3,5011,560,1,1,1270.0,2008.0
4,6102,612,1,1,1270.0,2008.0
...,...,...,...,...,...,...
1017204,4771,339,1,1,5350.0,NaN
1017205,4540,326,1,1,5350.0,NaN
1017206,4297,300,1,1,5350.0,NaN
1017207,3697,305,1,1,5350.0,NaN


In [ ]:
outlier_vars(new_train, show_plot=True)

In [20]:
new_train.StateHoliday.value_counts()

0    855087
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

In [21]:
new_test.StateHoliday.value_counts()

0    40908
a      180
Name: StateHoliday, dtype: int64

> combine the `0` values together

In [22]:
new_train.StateHoliday = new_train.StateHoliday.str.strip()

In [23]:
new_train.StateHoliday.value_counts()

0    855087
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

> convert categorical columns  in new train

convert to object dtype

In [24]:
cols = ['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday',
       'StoreType','Assortment','CompetitionOpenSinceMonth','Promo2', 'Date']

In [25]:
new_train=change_datatype(new_train, dtype='object', col_list=cols,date_col_name='Date')

In [28]:
new_test=change_datatype(new_test, dtype='object', col_list=cols,date_col_name='Date')

In [27]:
new_train.dtypes

Store                                 int64
DayOfWeek                            object
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                 object
Promo                                object
StateHoliday                         object
SchoolHoliday                        object
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth            object
CompetitionOpenSinceYear            float64
Promo2                               object
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

In [29]:
new_test.dtypes

Id                                    int64
Store                                 int64
DayOfWeek                            object
Date                         datetime64[ns]
Open                                 object
Promo                                object
StateHoliday                         object
SchoolHoliday                        object
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth            object
CompetitionOpenSinceYear            float64
Promo2                               object
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

In [30]:
new_train.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                 131072
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

In [32]:
new_test.isnull().sum()

Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                            11
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment                       0
CompetitionDistance             96
CompetitionOpenSinceMonth    15216
CompetitionOpenSinceYear     15216
Promo2                           0
Promo2SinceWeek              17232
Promo2SinceYear              17232
PromoInterval                17232
dtype: int64

> Extracting Date Columns

In [80]:
new_train = ex

In [37]:
new_test['Season'].value_counts()

Summer    26536
Fall      14552
Name: Season, dtype: int64

In [38]:
new_train['Season'].value_counts()

Spring    307740
Summer    262015
Winter    260904
Fall      186550
Name: Season, dtype: int64

> FIll Na Values

In [66]:
new_train = fill_na(new_train)
new_test = fill_na(new_test)

In [67]:
new_train.isnull().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
Month                        0
Quarter                      0
Year                         0
Day                          0
Week                         0
Season                       0
dtype: int64

In [68]:
new_test.isnull().sum()

Id                           0
Store                        0
DayOfWeek                    0
Date                         0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
Month                        0
Quarter                      0
Year                         0
Day                          0
Week                         0
Season                       0
dtype: int64

In [70]:
new_train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Quarter,Year,Day,Week,Season
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,31,31,Summer
1,1,4,2015-07-30,5020,546,1,1,0,1,c,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,30,31,Summer
2,1,3,2015-07-29,4782,523,1,1,0,1,c,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,29,31,Summer
3,1,2,2015-07-28,5011,560,1,1,0,1,c,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,28,31,Summer
4,1,1,2015-07-27,6102,612,1,1,0,1,c,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,27,31,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,5,1,Winter
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,4,1,Winter
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,3,1,Winter
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,2,1,Winter


> call pipeline function to preprocess data

In [72]:
path = '../../datasets/processed/' #to_drop=["Date"],

In [73]:
new_train1 = new_train.drop(columns=['Date'], axis=1)

In [74]:
new_train1

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Quarter,Year,Day,Week,Season
0,1,5,5263,555,1,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,31,31,Summer
1,1,4,5020,546,1,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,30,31,Summer
2,1,3,4782,523,1,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,29,31,Summer
3,1,2,5011,560,1,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,28,31,Summer
4,1,1,6102,612,1,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",7,3,2015,27,31,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,4771,339,1,0,0,1,d,c,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,5,1,Winter
1017205,1115,5,4540,326,1,0,0,1,d,c,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,4,1,Winter
1017206,1115,4,4297,300,1,0,0,1,d,c,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,3,1,Winter
1017207,1115,3,3697,305,1,0,0,1,d,c,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,1,2013,2,1,Winter


In [75]:
prpcsd_data = preprocess(new_train1, save_path=path, obj_name='trans_data.pkl')

In [76]:
prpcsd_data

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Quarter,Year,Day,Week,Season
0,-0.998208,0.25,-0.116493,-0.125,0,1,1,-0.171799,0.333333,-0.666667,...,0.2,0.5,1,1,0.375,0,c,a,"Jan,Apr,Jul,Oct",Summer
1,-0.998208,0,-0.175345,-0.145833,0,1,1,-0.171799,0.333333,-0.666667,...,0.2,0.5,1,0.933333,0.375,0,c,a,"Jan,Apr,Jul,Oct",Summer
2,-0.998208,-0.25,-0.232986,-0.199074,0,1,1,-0.171799,0.333333,-0.666667,...,0.2,0.5,1,0.866667,0.375,0,c,a,"Jan,Apr,Jul,Oct",Summer
3,-0.998208,-0.5,-0.177525,-0.113426,0,1,1,-0.171799,0.333333,-0.666667,...,0.2,0.5,1,0.8,0.375,0,c,a,"Jan,Apr,Jul,Oct",Summer
4,-0.998208,-0.75,0.0867038,0.00694444,0,1,1,-0.171799,0.333333,-0.666667,...,0.2,0.5,1,0.733333,0.375,0,c,a,"Jan,Apr,Jul,Oct",Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,0.998208,0.5,-0.23565,-0.625,0,0,1,0.489465,0,0,...,-1,-0.5,-1,-0.733333,-0.875,0,d,c,"Mar,Jun,Sept,Dec",Winter
1017205,0.998208,0.25,-0.291596,-0.655093,0,0,1,0.489465,0,0,...,-1,-0.5,-1,-0.8,-0.875,0,d,c,"Mar,Jun,Sept,Dec",Winter
1017206,0.998208,0,-0.350448,-0.715278,0,0,1,0.489465,0,0,...,-1,-0.5,-1,-0.866667,-0.875,0,d,c,"Mar,Jun,Sept,Dec",Winter
1017207,0.998208,-0.25,-0.495762,-0.703704,0,0,1,0.489465,0,0,...,-1,-0.5,-1,-0.933333,-0.875,0,d,c,"Mar,Jun,Sept,Dec",Winter


In [77]:
prpcsd_data.dtypes

Store                        object
DayOfWeek                    object
Sales                        object
Customers                    object
Open                         object
Promo                        object
StateHoliday                 object
SchoolHoliday                object
StoreType                    object
Assortment                   object
CompetitionDistance          object
CompetitionOpenSinceMonth    object
CompetitionOpenSinceYear     object
Promo2                       object
Promo2SinceWeek              object
Promo2SinceYear              object
PromoInterval                object
Month                        object
Quarter                      object
Year                         object
Day                          object
Week                         object
Season                       object
dtype: object

In [25]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


> save preprocessed data

> Build Pipeline for Models

The following SKLearn estimators would be used:

- Random Forest Regressors

- Xgboost

- 

In [26]:
sm = SimpleImputer(strategy='most_frequent', missing_values=np.nan)